# Autoencoder

Autoencoders learn how to compress and decompress complex data like images. This is conceptually similar to PCA-based dimensionality reduction, but autoencoders can capture nonlinear patterns, whereas PCA is limited to linear relationships.

This is an example of **unsupervised** DL.

<img src="https://contenthub-static.grammarly.com/blog/wp-content/uploads/2024/10/6303_blog-visuals-auto-encoders_1500X800.png" width=800>

The Autoencoder structure shows:

- **Input Data** ➔ **Encoder Layers** ➔ **Latent Space (Bottleneck)** ➔ **Decoder Layers** ➔ **Reconstructed Data**

| Part | Diagram Description | Code (Implementation) |
|:---|:---|:---|
| **Input Layer** | Green nodes on the left (Input Data) | `Input(shape=(4096,))` |
| **Encoder Layers** | Left side layers compressing the input | `Dense(512, relu)`, `Dense(128, relu)` |
| **Bottleneck (Latent Space)** | Red nodes (smallest point) | `Dense(128)` |
| **Decoder Layers** | Right side layers expanding back | `Dense(512, relu)`, `Dense(4096, sigmoid)` |
| **Output Layer** | Final reconstructed output | `Model(input_img, decoded)` |


---


- The **Encoder** compresses a 4096-dimensional input (64×64 face image) into a **128-dimensional feature vector**.
- The **Bottleneck** is the compressed representation that holds essential information about the face.
- The **Decoder** expands the 128 features back to reconstruct the original 4096 features.
- The model is trained to **minimize the difference** between the input and reconstructed output.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_olivetti_faces
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# Load Olivetti Faces Dataset
faces_data = fetch_olivetti_faces(shuffle=True, random_state=42)
faces = faces_data.data  # shape: (400, 4096)
targets = faces_data.target  # not used here

In datasets like MNIST or Olivetti Faces (after normalization), pixel values are floating numbers from 0 to 1

In [ ]:
print(f"Faces shape: {faces.shape}")  # 400 images, each 64x64 (flattened to 4096)

In [ ]:
# Create a DataFrame with the face data

# First, create a DataFrame with the pixel values
pixel_columns = [f'pixel_{i}' for i in range(faces.shape[1])]
faces_df = pd.DataFrame(faces, columns=pixel_columns)

# Add the target (person identifier) as a column
faces_df['person_id'] = targets

print("First 5 rows of the DataFrame:")
faces_df.head()

In [ ]:
# Split into train and test
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(faces, test_size=0.2, random_state=42)

Step 1: Define the architecture (Autoencoder)

In [ ]:
# Input Layer: 4096 neurons, representing the 64x64 face images flattened into a vector
input_img = tf.keras.Input(shape=(faces.shape[1],))

# Encoder Part:
# First hidden layer with 512 neurons and ReLU activation
encoded = layers.Dense(512, activation='relu')(input_img)

# Second hidden layer with 128 neurons (bottleneck/latent space), compressing features further
encoded = layers.Dense(256, activation='relu')(encoded)

# Decoder Part:
# First decoding layer expands back to 512 neurons with ReLU activation
decoded = layers.Dense(512, activation='relu')(encoded)

# Output layer: reconstructs the original 4096-dimensional image using sigmoid activation
decoded = layers.Dense(4096, activation='sigmoid')(decoded)

# Define the complete Autoencoder model connecting input to reconstructed output
autoencoder = models.Model(input_img, decoded)

In [ ]:
# Compile the model with Adam optimizer and binary cross-entropy loss
autoencoder.compile(optimizer='adam',
                    loss='mse')

In [ ]:
# Train Autoencoder
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Reconstruct faces
decoded_faces = autoencoder.predict(x_test)

In [ ]:
# Plot first 12 Original and Reconstructed faces
n = 12
plt.figure(figsize=(20, 4))
for i in range(n):
    # Original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(64, 64), cmap='gray')
    plt.title("Original")
    plt.axis('off')

    # Reconstructed
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_faces[i].reshape(64, 64), cmap='gray')
    plt.title("Reconstructed")
    plt.axis('off')

plt.show()

| PCA | Autoencoder |
|:---|:---|
| Very strong for linear patterns | Can capture nonlinear patterns (but needs tuning!) |
| May outperform basic autoencoder on structured datasets | Needs better architecture + training to shine |
| No training time | Requires training and optimization |
